## Primera tarea de codificación CC-571

El siguiente programada, llamado `process-run.py`,  permite ver cómo cambia el estado de un proceso a medida que se ejecuta en una CPU.

In [1]:
%%writefile process-run.py

#! /usr/bin/env python3

import sys
from optparse import OptionParser
import random

SCHED_SWITCH_ON_IO = 'SWITCH_ON_IO'
SCHED_SWITCH_ON_END = 'SWITCH_ON_END'

IO_RUN_LATER = 'IO_RUN_LATER'
IO_RUN_IMMEDIATE = 'IO_RUN_IMMEDIATE'

STATE_RUNNING = 'RUNNING'
STATE_READY = 'READY'
STATE_DONE = 'DONE'
STATE_WAIT = 'WAITING'

PROC_CODE = 'code_'
PROC_PC = 'pc_'
PROC_ID = 'pid_'
PROC_STATE = 'proc_state_'

DO_COMPUTE = 'cpu'
DO_IO = 'io'


class scheduler:
    def __init__(self, process_switch_behavior, io_done_behavior, io_length):
        self.proc_info = {}
        self.process_switch_behavior = process_switch_behavior
        self.io_done_behavior = io_done_behavior
        self.io_length = io_length
        return

    def new_process(self):
        proc_id = len(self.proc_info)
        self.proc_info[proc_id] = {}
        self.proc_info[proc_id][PROC_PC] = 0
        self.proc_info[proc_id][PROC_ID] = proc_id
        self.proc_info[proc_id][PROC_CODE] = []
        self.proc_info[proc_id][PROC_STATE] = STATE_READY
        return proc_id

    def load_file(self, progfile):
        fd = open(progfile)
        proc_id = self.new_process()
        
        for line in fd:
            tmp = line.split()
            if len(tmp) == 0:
                continue
            opcode = tmp[0]
            if opcode == 'compute':
                assert(len(tmp) == 2)
                for i in range(int(tmp[1])):
                    self.proc_info[proc_id][PROC_CODE].append(DO_COMPUTE)
            elif opcode == 'io':
                assert(len(tmp) == 1)
                self.proc_info[proc_id][PROC_CODE].append(DO_IO)
        fd.close()
        return

    def load(self, program_description):
        proc_id = self.new_process()
        tmp = program_description.split(':')
        if len(tmp) != 2:
            print ('Bad description (%s): Must be number <x:y>' % program_description)
            print ('  where X is the number of instructions')
            print ('  and Y is the percent change that an instruction is CPU not IO')
            exit(1)

        num_instructions, chance_cpu = int(tmp[0]), float(tmp[1])/100.0
        for i in range(num_instructions):
            if random.random() < chance_cpu: # aquí se determina el tipo de instrucción.
                self.proc_info[proc_id][PROC_CODE].append(DO_COMPUTE)
            else:
                self.proc_info[proc_id][PROC_CODE].append(DO_IO)
        return

    def move_to_ready(self, expected, pid=-1):
        if pid == -1:
            pid = self.curr_proc
        assert(self.proc_info[pid][PROC_STATE] == expected)
        self.proc_info[pid][PROC_STATE] = STATE_READY
        return

    def move_to_wait(self, expected):
        assert(self.proc_info[self.curr_proc][PROC_STATE] == expected)
        self.proc_info[self.curr_proc][PROC_STATE] = STATE_WAIT
        return

    def move_to_running(self, expected):
        assert(self.proc_info[self.curr_proc][PROC_STATE] == expected)
        self.proc_info[self.curr_proc][PROC_STATE] = STATE_RUNNING
        return

    def move_to_done(self, expected):
        assert(self.proc_info[self.curr_proc][PROC_STATE] == expected)
        self.proc_info[self.curr_proc][PROC_STATE] = STATE_DONE
        return

    def next_proc(self, pid=-1):
        if pid != -1:
            self.curr_proc = pid
            self.move_to_running(STATE_READY)
            return
        for pid in range(self.curr_proc + 1, len(self.proc_info)):
            if self.proc_info[pid][PROC_STATE] == STATE_READY:
                self.curr_proc = pid
                self.move_to_running(STATE_READY)
                return
        for pid in range(0, self.curr_proc + 1):
            if self.proc_info[pid][PROC_STATE] == STATE_READY:
                self.curr_proc = pid
                self.move_to_running(STATE_READY)
                return
        return

    def get_num_processes(self):
        return len(self.proc_info)

    def get_num_instructions(self, pid):
        return len(self.proc_info[pid][PROC_CODE])

    def get_instruction(self, pid, index):
        return self.proc_info[pid][PROC_CODE][index]

    def get_num_active(self):
        num_active = 0
        for pid in range(len(self.proc_info)):
            if self.proc_info[pid][PROC_STATE] != STATE_DONE:
                num_active += 1
        return num_active

    def get_num_runnable(self):
        num_active = 0
        for pid in range(len(self.proc_info)):
            if self.proc_info[pid][PROC_STATE] == STATE_READY or \
                   self.proc_info[pid][PROC_STATE] == STATE_RUNNING:
                num_active += 1
        return num_active

    def get_ios_in_flight(self, current_time):
        num_in_flight = 0
        for pid in range(len(self.proc_info)):
            for t in self.io_finish_times[pid]:
                if t > current_time:
                    num_in_flight += 1
        return num_in_flight

    def check_for_switch(self):
        return

    def space(self, num_columns):
        for i in range(num_columns):
            print ('%10s' % ' ', end='')

    def check_if_done(self):
        if len(self.proc_info[self.curr_proc][PROC_CODE]) == 0:
            if self.proc_info[self.curr_proc][PROC_STATE] == STATE_RUNNING:
                self.move_to_done(STATE_RUNNING)
                self.next_proc()
        return

    def run(self):
        clock_tick = 0

        if len(self.proc_info) == 0:
            return

        self.io_finish_times = {}
        for pid in range(len(self.proc_info)):
            self.io_finish_times[pid] = []

        self.curr_proc = 0
        self.move_to_running(STATE_READY)

        print ('%s' % 'Time', end='') 
        for pid in range(len(self.proc_info)):
            print ('%10s' % ('PID:%2d' % (pid)), end='')
        print ('%10s' % 'CPU', end='')
        print ('%10s' % 'IOs', end='')
        print ('')

        io_busy = 0
        cpu_busy = 0

        while self.get_num_active() > 0:
            clock_tick += 1

            io_done = False
            for pid in range(len(self.proc_info)):
                if clock_tick in self.io_finish_times[pid]:
                    io_done = True
                    self.move_to_ready(STATE_WAIT, pid)
                    if self.io_done_behavior == IO_RUN_IMMEDIATE:
                        if self.curr_proc != pid:
                            if self.proc_info[self.curr_proc][PROC_STATE] == STATE_RUNNING:
                                self.move_to_ready(STATE_RUNNING)
                        self.next_proc(pid)
                    else:
                        if self.process_switch_behavior == SCHED_SWITCH_ON_END and self.get_num_runnable() > 1:
                            self.next_proc(pid)
                        if self.get_num_runnable() == 1:
                            self.next_proc(pid)
                    self.check_if_done()
            
            instruction_to_execute = ''
            if self.proc_info[self.curr_proc][PROC_STATE] == STATE_RUNNING and \
                   len(self.proc_info[self.curr_proc][PROC_CODE]) > 0:
                instruction_to_execute = self.proc_info[self.curr_proc][PROC_CODE].pop(0)
                cpu_busy += 1

            if io_done:
                print ('%3d*' % clock_tick,end='')
            else:
                print ('%3d ' % clock_tick,end='')
            for pid in range(len(self.proc_info)):
                if pid == self.curr_proc and instruction_to_execute != '':
                    print ('%10s' % ('RUN:'+instruction_to_execute), end='')
                else:
                    print ('%10s' % (self.proc_info[pid][PROC_STATE]),end='')
            if instruction_to_execute == '':
                print ('%10s' % ' ', end='')
            else:
                print ('%10s' % 1, end='')
            num_outstanding = self.get_ios_in_flight(clock_tick)
            if num_outstanding > 0:
                print ('%10s' % str(num_outstanding),end='')
                io_busy += 1
            else:
                print ('%10s' % ' ', end='')
            print ('')

            if instruction_to_execute == DO_IO:
                self.move_to_wait(STATE_RUNNING)
                self.io_finish_times[self.curr_proc].append(clock_tick + self.io_length)
                if self.process_switch_behavior == SCHED_SWITCH_ON_IO:
                    self.next_proc()

            self.check_if_done()
        return (cpu_busy, io_busy, clock_tick)
        


parser = OptionParser()
parser.add_option('-s', '--seed', default=0, help='the random seed', action='store', type='int', dest='seed')
parser.add_option('-l', '--processlist', default='',
                  help='a comma-separated list of processes to run, in the form X1:Y1,X2:Y2,... where X is the number of instructions that process should run, and Y the chances (from 0 to 100) that an instruction will use the CPU or issue an IO',
                  action='store', type='string', dest='process_list')
parser.add_option('-L', '--iolength', default=5, help='how long an IO takes', action='store', type='int', dest='io_length')
parser.add_option('-S', '--switch', default='SWITCH_ON_IO',
                  help='when to switch between processes: SWITCH_ON_IO, SWITCH_ON_END',
                  action='store', type='string', dest='process_switch_behavior')
parser.add_option('-I', '--iodone', default='IO_RUN_LATER',
                  help='type of behavior when IO ends: IO_RUN_LATER, IO_RUN_IMMEDIATE',
                  action='store', type='string', dest='io_done_behavior')
parser.add_option('-c', help='compute answers for me', action='store_true', default=False, dest='solve')
parser.add_option('-p', '--printstats', help='print statistics at end; only useful with -c flag (otherwise stats are not printed)', action='store_true', default=False, dest='print_stats')
(options, args) = parser.parse_args()

random.seed(options.seed)

assert(options.process_switch_behavior == SCHED_SWITCH_ON_IO or \
       options.process_switch_behavior == SCHED_SWITCH_ON_END)
assert(options.io_done_behavior == IO_RUN_IMMEDIATE or \
       options.io_done_behavior == IO_RUN_LATER)

s = scheduler(options.process_switch_behavior, options.io_done_behavior, options.io_length)

for p in options.process_list.split(','):
    s.load(p)

if options.solve == False:
    print ('Produce a trace of what would happen when you run these processes:')
    for pid in range(s.get_num_processes()):
        print ('Process %d' % pid)
        for inst in range(s.get_num_instructions(pid)):
            print ('  %s' % s.get_instruction(pid, inst))
        print ('')
    print ('Important behaviors:')
    print ('  System will switch when'),
    if options.process_switch_behavior == SCHED_SWITCH_ON_IO:
        print ('the current process is FINISHED or ISSUES AN IO')
    else:
        print ('the current process is FINISHED')
    print ('  After IOs, the process issuing the IO will'),
    if options.io_done_behavior == IO_RUN_IMMEDIATE:
        print ('run IMMEDIATELY')
    else:
        print ('run LATER (when it is its turn)')
    print ('')
    exit(0)

(cpu_busy, io_busy, clock_tick) = s.run()

if options.print_stats:
    print ('')
    print ('Stats: Total Time %d' % clock_tick)
    print ('Stats: CPU Busy %d (%.2f%%)' % (cpu_busy, 100.0 * float(cpu_busy)/clock_tick))
    print ('Stats: IO Busy  %d (%.2f%%)' % (io_busy, 100.0 * float(io_busy)/clock_tick))
    print ('')

Overwriting process-run.py


#### Ejercicio 1 

Corrige y ejecuta el script  y muestra las distintas opciones que produce.

##### Respuesta 1:

In [2]:
!python3 process-run.py -h

Usage: process-run.py [options]

Options:
  -h, --help            show this help message and exit
  -s SEED, --seed=SEED  the random seed
  -l PROCESS_LIST, --processlist=PROCESS_LIST
                        a comma-separated list of processes to run, in the
                        form X1:Y1,X2:Y2,... where X is the number of
                        instructions that process should run, and Y the
                        chances (from 0 to 100) that an instruction will use
                        the CPU or issue an IO
  -L IO_LENGTH, --iolength=IO_LENGTH
                        how long an IO takes
  -S PROCESS_SWITCH_BEHAVIOR, --switch=PROCESS_SWITCH_BEHAVIOR
                        when to switch between processes: SWITCH_ON_IO,
                        SWITCH_ON_END
  -I IO_DONE_BEHAVIOR, --iodone=IO_DONE_BEHAVIOR
                        type of behavior when IO ends: IO_RUN_LATER,
                        IO_RUN_IMMEDIATE
  -c                    compute answers for me
  -p, --prints

#### Ejercicio 2

Explica que función realizan las siguientes opciones:

- `PROCESS_LIST`:
- `PROCESS_SWITCH_BEHAVIOR`:
- `IO_DONE_BEHAVIOR`:

##### Respuesta 2:
- `PROCESS_LIST`:
pide que ingreses una lista de procesos separados por comas. 
El proceso está definido por una dupla X:Y donde X es el número de instrucciones
que tendrá el proceso y Y es la probabilidad de que una instrucción use la CPU. Es decir,
si se tiene probabilidad Y de usar CPU se tendrá 100 - Y de usar el dispositivo I/O
- `PROCESS_SWITCH_BEHAVIOR`:
indica la forma en que se altenernará la ejecución entre procesos. Hay dos opciones, 
la primera y la segunda. Cambiar con cada interrupción I/O o cambiar cuando un proceso finalice.
- `IO_DONE_BEHAVIOR`:
determina si el proceso se retomará o no inmediatamente luego de que este haya usado 
el dispositivo de entrada y salida. Si se selecciona IO_RUN_LATER, entonces el proceso se
añadirá a la cola y tendrá que esperar su turno para reanudarse.

#### Ejercicio 3:

Explica las salidas de (en colab o jupyter usar una comando mágico como `%run file.py`): 

- `prompt> ./process-run.py -l 5:100`

- `prompt> ./process-run.py -l 5:100 -c`

- `prompt> ./process-run.py -l 5:100,5:100 `

- `prompt> ./process-run.py -l 5:100,5:100 -c`

- `prompt> ./process-run.py -l 3:0 -L 5`

- `prompt> ./process-run.py -l 3:0 -L 5 -c`

##### Respuesta 3:
Veamos

In [3]:
!python3 process-run.py -l 5:100

Produce a trace of what would happen when you run these processes:
Process 0
  cpu
  cpu
  cpu
  cpu
  cpu

Important behaviors:
  System will switch when
the current process is FINISHED or ISSUES AN IO
  After IOs, the process issuing the IO will
run LATER (when it is its turn)



Muestra la secuencia de instrucciones que tendría el proceso 0. 
Podemos ver que usaría 5 veces el cpu, tal y como lo especificamos en la entrada.
También nos alerta sobre el modo de cambio de procesos y la forma en que
los procesos son retomados después de una interrupción de I/

In [4]:
!python3 process-run.py -l 5:100 -c

Time    PID: 0       CPU       IOs
  1    RUN:cpu         1          
  2    RUN:cpu         1          
  3    RUN:cpu         1          
  4    RUN:cpu         1          
  5    RUN:cpu         1          


Muestra la simulación del proceso 0. Podemos ver que hay 5 pasos, como se ha especificado en la entrada.
Los 5 pasos piden el CPU y no se pide entrada y salida en ningún momento.

In [5]:
!python3 process-run.py -l 5:100,5:100

Produce a trace of what would happen when you run these processes:
Process 0
  cpu
  cpu
  cpu
  cpu
  cpu

Process 1
  cpu
  cpu
  cpu
  cpu
  cpu

Important behaviors:
  System will switch when
the current process is FINISHED or ISSUES AN IO
  After IOs, the process issuing the IO will
run LATER (when it is its turn)



Muestra las instrucciones definidas para dos procesos identificados como uno y dos.
Como las veces anterior, nos alerta sobre el modo de cambio de procesos y la 
forma en que los procesos son retomados después de una interrupción de I/O

In [6]:
!python3 process-run.py -l 5:100,5:100 -c

Time    PID: 0    PID: 1       CPU       IOs
  1    RUN:cpu     READY         1          
  2    RUN:cpu     READY         1          
  3    RUN:cpu     READY         1          
  4    RUN:cpu     READY         1          
  5    RUN:cpu     READY         1          
  6       DONE   RUN:cpu         1          
  7       DONE   RUN:cpu         1          
  8       DONE   RUN:cpu         1          
  9       DONE   RUN:cpu         1          
 10       DONE   RUN:cpu         1          


Muestra la simulación de la programación anterior. El proceso 1 espera a que el proceso 0 termine de
desocupar el cpu. Luego, corre. No se usa I/O.

In [7]:
!python3 process-run.py -l 3:0 -L 5

Produce a trace of what would happen when you run these processes:
Process 0
  io
  io
  io

Important behaviors:
  System will switch when
the current process is FINISHED or ISSUES AN IO
  After IOs, the process issuing the IO will
run LATER (when it is its turn)



Muestra las instrucciones del proceso 0. Se ha puesto el tiempo que tarda una interrupción I/O a 5,
pero no se ha puesto en acción. De hecho, ese es el valor por defecto.

In [8]:
!python3 process-run.py -l 3:0 L=5 -c

Time    PID: 0       CPU       IOs
  1     RUN:io         1          
  2    WAITING                   1
  3    WAITING                   1
  4    WAITING                   1
  5    WAITING                   1
  6*    RUN:io         1          
  7    WAITING                   1
  8    WAITING                   1
  9    WAITING                   1
 10    WAITING                   1
 11*    RUN:io         1          
 12    WAITING                   1
 13    WAITING                   1
 14    WAITING                   1
 15    WAITING                   1
 16*      DONE                    


Realiza la simulación. El proceso realiza tres operaciones de I/O. Se tarda lo esperado 15 pasos.

#### Ejercicio 4:

- Imprime estadísticas usando los mismo comandos anteriores, pero con la opción `-p`

- Explica tus conclusiones.

##### Respuesta 4:
Veamos:

In [9]:
!python3 process-run.py -l 5:100 -c -p
print('---------------------------------------------')
!python3 process-run.py -l 5:100,5:100 -c -p
print('---------------------------------------------')
!python3 process-run.py -l 3:0 L=5 -c -p

Time    PID: 0       CPU       IOs
  1    RUN:cpu         1          
  2    RUN:cpu         1          
  3    RUN:cpu         1          
  4    RUN:cpu         1          
  5    RUN:cpu         1          

Stats: Total Time 5
Stats: CPU Busy 5 (100.00%)
Stats: IO Busy  0 (0.00%)

---------------------------------------------
Time    PID: 0    PID: 1       CPU       IOs
  1    RUN:cpu     READY         1          
  2    RUN:cpu     READY         1          
  3    RUN:cpu     READY         1          
  4    RUN:cpu     READY         1          
  5    RUN:cpu     READY         1          
  6       DONE   RUN:cpu         1          
  7       DONE   RUN:cpu         1          
  8       DONE   RUN:cpu         1          
  9       DONE   RUN:cpu         1          
 10       DONE   RUN:cpu         1          

Stats: Total Time 10
Stats: CPU Busy 10 (100.00%)
Stats: IO Busy  0 (0.00%)

---------------------------------------------
Time    PID: 0       CPU       IOs
  1     RUN:io

En las dos primeras simulaciones, el cpu se usa al 100% y IO se usa al 0%. 
En cambio, en la tercera simulación el CPU se usa al 18.75% y IO se usa al 75%.
Es importante notar que el CPU se usa al momento de la interrupción I/O, por lo tanto,
para iniciar IO se necesitará primero el CPU libre.
La diferencia en el porcentaje, puesto que observamos que %CPU y %IO no suman 100, se encuentra en el tiempo 16, que no se usa para nada
pero se cuenta en la estadística..

#### Ejercicio 5:

Una opción importante es `-S`. Con esta opción configurada en `SWITCH_ON_END`, el sistema NO cambiará a otro proceso mientras uno esté realizando un `E/S`, sino que esperará hasta que el proceso haya terminado por completo.

- Explica que hace la opción `-S`
- ¿Qué sucede cuando se ejecutan los siguientes dos procesos (`-l 1:0,4:100 -c -S SWITCH_ON_END`), uno haciendo `E/S` y el otro haciendo trabajo de CPU?


##### Respuesta 5:
+ La opción `-S` modifica la forma en que se ejecutan los procesos.

Ahora, realicemos la simulación:

In [10]:
!python3 process-run.py -l 1:0,4:100 -c -S SWITCH_ON_END -p

Time    PID: 0    PID: 1       CPU       IOs
  1     RUN:io     READY         1          
  2    WAITING     READY                   1
  3    WAITING     READY                   1
  4    WAITING     READY                   1
  5    WAITING     READY                   1
  6*      DONE   RUN:cpu         1          
  7       DONE   RUN:cpu         1          
  8       DONE   RUN:cpu         1          
  9       DONE   RUN:cpu         1          

Stats: Total Time 9
Stats: CPU Busy 5 (55.56%)
Stats: IO Busy  4 (44.44%)



+ Una vez iniciada la ejecución el proceso 1 se queda esperando a que el proceso 0 termine para empezar a usar el CPU. 
Esto es un desproposito puesto que, según nuestro modelo, el CPU y el dispositivo I/O pueden
operar en paralelo. Note que el CPU se usa al 55.56% y IO al 44.44%

#### Ejercicio 6: 

Ahora, ejecute los mismos procesos, pero con el comportamiento `switching` configurado para cambiar a otro proceso siempre `WAITING` es para `E/S` (`-l 1:0,4:100 -c -S SWITCH_ON_IO`). 

- ¿Que pasa ahora? 
- Utiliza `-c` y `-p` y explica que ocurre.

##### Respuesta 7:
Ejecutemos lo solicitado:

In [11]:
!python3 process-run.py -l 1:0,4:100 -c -S SWITCH_ON_IO

Time    PID: 0    PID: 1       CPU       IOs
  1     RUN:io     READY         1          
  2    WAITING   RUN:cpu         1         1
  3    WAITING   RUN:cpu         1         1
  4    WAITING   RUN:cpu         1         1
  5    WAITING   RUN:cpu         1         1
  6*      DONE      DONE                    


+ Ahora, cada vez que hay una interrupción IO, se cambia el proceso que se está ejecutando.

In [12]:
!python3 process-run.py -l 1:0,4:100 -c -S SWITCH_ON_IO -p

Time    PID: 0    PID: 1       CPU       IOs
  1     RUN:io     READY         1          
  2    WAITING   RUN:cpu         1         1
  3    WAITING   RUN:cpu         1         1
  4    WAITING   RUN:cpu         1         1
  5    WAITING   RUN:cpu         1         1
  6*      DONE      DONE                    

Stats: Total Time 6
Stats: CPU Busy 5 (83.33%)
Stats: IO Busy  4 (66.67%)



+ El tiempo de ejecución se ve tremendamente reducido.

#### Ejercicio 7:

Otro comportamiento importante es qué hacer cuando se completa una `E/S`. 


- ¿Qué sucede  con  `-I IO_RUN_LATER`, cuando se completa una `E/S`
- Qué sucede cuando ejecuta esta combinación de procesos: 
 ` prompt> ./process-run.py -l 3:0,5:100,5:100,5:100 -S SWITCH_ON_IO -I IO_RUN_LATER -c -p`).  ¿Se están utilizando eficazmente los recursos del sistema?.

- Ejecuta los mismos procesos, pero con con `-I IO RUN IMMEDIATE`, ¿cómo difiere este comportamiento del anterior?. ¿Por qué crees que podría ser una buena idea ejecutar un proceso que acaba de completar una `E/S` nuevamente?.

##### Respuesta 7:
+ Con la bandera `-I IO_RUN_LATER`, el proceso que llamó a la IO es puesto en la cola para reanudarse cuando su turno llegue nuevamente.

In [13]:
!python3 process-run.py -l 3:0,5:100,5:100,5:100 -S SWITCH_ON_IO -I IO_RUN_LATER -c -p

Time    PID: 0    PID: 1    PID: 2    PID: 3       CPU       IOs
  1     RUN:io     READY     READY     READY         1          
  2    WAITING   RUN:cpu     READY     READY         1         1
  3    WAITING   RUN:cpu     READY     READY         1         1
  4    WAITING   RUN:cpu     READY     READY         1         1
  5    WAITING   RUN:cpu     READY     READY         1         1
  6*     READY   RUN:cpu     READY     READY         1          
  7      READY      DONE   RUN:cpu     READY         1          
  8      READY      DONE   RUN:cpu     READY         1          
  9      READY      DONE   RUN:cpu     READY         1          
 10      READY      DONE   RUN:cpu     READY         1          
 11      READY      DONE   RUN:cpu     READY         1          
 12      READY      DONE      DONE   RUN:cpu         1          
 13      READY      DONE      DONE   RUN:cpu         1          
 14      READY      DONE      DONE   RUN:cpu         1          
 15      READY      DONE 

+ No se utiliza eficazmente los recursos del sistema. Note que el proceso 0 es el único
que utiliza IO. Sin embargo, esta no puede concretarse porque, debido la bandera SWITCH_ON_IO
tiene que esperar al final de la cola para poder utilizar el CPU para realizar la 
interrupción cuando IO podría utilizarse mientras el resto de procesos sigue usando el 
CPU luego de que se hubiese llamado a la interrupción.

In [14]:
!python3 process-run.py -l 3:0,5:100,5:100,5:100 -S SWITCH_ON_IO -I IO_RUN_IMMEDIATE -c -p

Time    PID: 0    PID: 1    PID: 2    PID: 3       CPU       IOs
  1     RUN:io     READY     READY     READY         1          
  2    WAITING   RUN:cpu     READY     READY         1         1
  3    WAITING   RUN:cpu     READY     READY         1         1
  4    WAITING   RUN:cpu     READY     READY         1         1
  5    WAITING   RUN:cpu     READY     READY         1         1
  6*    RUN:io     READY     READY     READY         1          
  7    WAITING   RUN:cpu     READY     READY         1         1
  8    WAITING      DONE   RUN:cpu     READY         1         1
  9    WAITING      DONE   RUN:cpu     READY         1         1
 10    WAITING      DONE   RUN:cpu     READY         1         1
 11*    RUN:io      DONE     READY     READY         1          
 12    WAITING      DONE   RUN:cpu     READY         1         1
 13    WAITING      DONE   RUN:cpu     READY         1         1
 14    WAITING      DONE      DONE   RUN:cpu         1         1
 15    WAITING      DONE 

+ Ahora, se ha hecho lo que indicamos en el caso anterior. Cada vez que se termina
una operación de entrada y salida, el proceso que la llamó vuelve a ocupar el CPU.
Esto mejora el rendimiento puesto que, una vez que se ha concretado la llamada a IO,
el CPU deja de ocuparse y pasa a ser usado por los demás procesos debido a la bandera
SWITCH_ON_IO. Sin embargo, recordemos que este es un caso particular en el cual solo 
uno de los procesos realiza únicamente IO y el resto únicamente CPU.

+ Para ilustrar esto, probemos lo siguiente: `-l 3:0,5:50,5:100,5:50 -S SWITCH_ON_IO -I IO_RUN_IMMEDIATE -c -p` donde, la mitad de las instrucciones de los procesos 1 y 3 serán IO.




In [15]:
!python3 process-run.py -l 3:0,5:50,5:100,5:50 -S SWITCH_ON_IO -I IO_RUN_IMMEDIATE -c -p

Time    PID: 0    PID: 1    PID: 2    PID: 3       CPU       IOs
  1     RUN:io     READY     READY     READY         1          
  2    WAITING   RUN:cpu     READY     READY         1         1
  3    WAITING    RUN:io     READY     READY         1         1
  4    WAITING   WAITING   RUN:cpu     READY         1         2
  5    WAITING   WAITING   RUN:cpu     READY         1         2
  6*    RUN:io   WAITING     READY     READY         1         1
  7    WAITING   WAITING   RUN:cpu     READY         1         2
  8*   WAITING   RUN:cpu     READY     READY         1         1
  9    WAITING    RUN:io     READY     READY         1         1
 10    WAITING   WAITING   RUN:cpu     READY         1         2
 11*    RUN:io   WAITING     READY     READY         1         1
 12    WAITING   WAITING   RUN:cpu     READY         1         2
 13    WAITING   WAITING      DONE    RUN:io         1         2
 14*   WAITING   RUN:cpu      DONE   WAITING         1         2
 15    WAITING      DONE 

+ Ahora, cambiemos el modo de ejecución después de la interrupción IO a `IO_RUN_LATER` con: `-l 3:0,5:100,5:100,5:100 -S SWITCH_ON_IO -I IO_RUN_LATER -c -p`

In [16]:
!python3 process-run.py -l 3:0,5:100,5:100,5:100 -S SWITCH_ON_IO -I IO_RUN_LATER -c -p

Time    PID: 0    PID: 1    PID: 2    PID: 3       CPU       IOs
  1     RUN:io     READY     READY     READY         1          
  2    WAITING   RUN:cpu     READY     READY         1         1
  3    WAITING   RUN:cpu     READY     READY         1         1
  4    WAITING   RUN:cpu     READY     READY         1         1
  5    WAITING   RUN:cpu     READY     READY         1         1
  6*     READY   RUN:cpu     READY     READY         1          
  7      READY      DONE   RUN:cpu     READY         1          
  8      READY      DONE   RUN:cpu     READY         1          
  9      READY      DONE   RUN:cpu     READY         1          
 10      READY      DONE   RUN:cpu     READY         1          
 11      READY      DONE   RUN:cpu     READY         1          
 12      READY      DONE      DONE   RUN:cpu         1          
 13      READY      DONE      DONE   RUN:cpu         1          
 14      READY      DONE      DONE   RUN:cpu         1          
 15      READY      DONE 

+ Note que el tiempo de ejecución disminuye significativamente. Así pues, esta política de ejecución no siempre será la más eficiente.
+ Una buena razón para que este tipo de ejecución sea una buena idea sería el hecho 
de que hay procesos que no siempre necesitan entrada y salida del usuario: alguno demonios.


#### Ejercicio 8:

Ejecuta con algunos procesos generados aleatoriamente: `-s 1 -l 3:50,3:50` o `-s 2 -l 3:50,3:50` o `-s 3 -l 3: 50,3: 50`. 

- Predice que resultará estas líneas. 
- ¿Qué sucede cuando  se usa la bandera `-I IO_RUN_INMEDIATE` vs. `-I IO_RUN_LATER` 
- ¿Qué sucede cuando se usa `-S SWITCH_ON_IO` vs. `-S SWITCH_ON_END?`.

##### Respuesta 8
+ Pues, el resultado del balance entre el número de instrucciones IO y CPU cambiará debido a que se ha cambiado el valor de la semilla para generar los números aleatorios que determinan el tipo de instrucción.

In [17]:
!python3 process-run.py -s 1 -l 3:50,3:50
print("-"*80)
!python3 process-run.py -s 2 -l 3:50,3:50
print("-"*80)
!python3 process-run.py -s 3 -l 3:50,3:50

Produce a trace of what would happen when you run these processes:
Process 0
  cpu
  io
  io

Process 1
  cpu
  cpu
  cpu

Important behaviors:
  System will switch when
the current process is FINISHED or ISSUES AN IO
  After IOs, the process issuing the IO will
run LATER (when it is its turn)

--------------------------------------------------------------------------------
Produce a trace of what would happen when you run these processes:
Process 0
  io
  io
  cpu

Process 1
  cpu
  io
  io

Important behaviors:
  System will switch when
the current process is FINISHED or ISSUES AN IO
  After IOs, the process issuing the IO will
run LATER (when it is its turn)

--------------------------------------------------------------------------------
Produce a trace of what would happen when you run these processes:
Process 0
  cpu
  io
  cpu

Process 1
  io
  io
  cpu

Important behaviors:
  System will switch when
the current process is FINISHED or ISSUES AN IO
  After IOs, the process issuin

In [18]:
!python3 process-run.py -s 1 -l 3:50,3:50 -I IO_RUN_IMMEDIATE
print('-'*80)
!python3 process-run.py -s 1 -l 3:50,3:50 -I IO_RUN_LATER

Produce a trace of what would happen when you run these processes:
Process 0
  cpu
  io
  io

Process 1
  cpu
  cpu
  cpu

Important behaviors:
  System will switch when
the current process is FINISHED or ISSUES AN IO
  After IOs, the process issuing the IO will
run IMMEDIATELY

--------------------------------------------------------------------------------
Produce a trace of what would happen when you run these processes:
Process 0
  cpu
  io
  io

Process 1
  cpu
  cpu
  cpu

Important behaviors:
  System will switch when
the current process is FINISHED or ISSUES AN IO
  After IOs, the process issuing the IO will
run LATER (when it is its turn)



+ Como se observa, hay una diferencia en la descripción de la salidas. En la primera, te alerta que el proceso que solicitó IO se reanudará INMEDIATAMENTE. En la segunda, te alerta que el proceso se reanudará DESPUÉS(cuando llegue su turno)

In [19]:
!python3 process-run.py -s 2 -l 3:50,3:50 -S SWITCH_ON_IO
print('-'*80)
!python3 process-run.py -s 3 -l 3:50,3:50 -S SWITCH_ON_END

Produce a trace of what would happen when you run these processes:
Process 0
  io
  io
  cpu

Process 1
  cpu
  io
  io

Important behaviors:
  System will switch when
the current process is FINISHED or ISSUES AN IO
  After IOs, the process issuing the IO will
run LATER (when it is its turn)

--------------------------------------------------------------------------------
Produce a trace of what would happen when you run these processes:
Process 0
  cpu
  io
  cpu

Process 1
  io
  io
  cpu

Important behaviors:
  System will switch when
the current process is FINISHED
  After IOs, the process issuing the IO will
run LATER (when it is its turn)



+ Como se observa, hay una diferencia en la descripción de la salidas. En la primera, se observa que el sistema cambiará el proceso en ejecución cuando el proceso actual haya FINALIZADO o cuando haya una LLAMADA A IO. En el segundo, te indica que que el sistema cambiará de proceso cuando el proceso actual haya FINALIZADO.